In [4]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

/home/akshay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [6]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [ ]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [ ]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [ ]:
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [ ]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

In [ ]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [ ]:
print(preds)
print(target)
print(loss.data)

In [1]:
x_share = (2,5,7)

In [2]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

def decrypt(shares):
    return sum(shares) % Q

In [3]:
shares = encrypt(3)
shares

(13007169585698468290914, 20507617513653401914298, 13966472588168608768237)

In [4]:
decrypt(shares)

3

In [5]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [6]:
x = encrypt(5)
y = encrypt(7)
z = add(x,y)
decrypt(z)

12

In [7]:
#Secret Sharing + Fixed Precision


bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [8]:
x = th.tensor([1,2,3,4,5])

In [9]:
x = x.share(bob, alice, secure_worker)

In [10]:
bob._objects

{91310985462: tensor([1802545707047959151,  357254481795170080, 1787067969408101250,
          452670523216913708,  916260572181753232])}

In [11]:
y=x+x

In [12]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:26885489298 -> bob:43704789929]
	-> (Wrapper)>[PointerTensor | me:33275424697 -> alice:13355212326]
	-> (Wrapper)>[PointerTensor | me:58050595157 -> secure_worker:98658554412]
	*crypto provider: me*

In [13]:
y.get()

tensor([ 2,  4,  6,  8, 10])

In [14]:
x = th.tensor([0.1,0.2,0.3])

In [15]:
x

tensor([0.1000, 0.2000, 0.3000])

In [16]:
x=x.fix_prec()

In [17]:
x.child.child

tensor([100, 200, 300])

In [18]:
y=x+x

In [19]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

In [20]:
x = th.tensor([0.1, 0.2, 0.3])

In [21]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [22]:
y=x+x

In [23]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])